In [6]:
%pip install yfinance scikit-learn matplotlib pandas


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from utils.SeriesProcessor import SeriesProcessor
from classes.frameworkDetector.framework_detector import FrameworkDetector

from classes.detectores.ADWINDetector import ADWINDetector
from classes.detectores.DDMDetector import DDMDetector
from classes.detectores.EDDMDetector import EDDMDetector
from classes.detectores.FHDDMDetector import FHDDMDetector
from classes.detectores.HDDM_ADetector import HDDM_ADetector
from classes.detectores.PageHinkleyDetector import PageHinkleyDetector
from classes.detectores.HDDM_WDetector import HDDM_WDetector
from classes.detectores.KSWINDetector import KSWINDetector

# **1. Preparação da Base de Dados**

In [2]:
# Lista de séries temporais disponíveis (para referência)
series = [
    "AAPL",
    "B3SA3.SA",
    "^IXIC",
    "^DJI",
    "^GSPC",
    "^BVSP",
    "USDBRL=X"
]

# Definindo parâmetros para processamento
lags = 5
serie_escolhida = series[-2]  # Exemplo: ^BVSP
print(f"Processando série: {serie_escolhida}")

# Baixando os dados
serie_temporal = SeriesProcessor.baixar_dados(serie_escolhida)

# Normalizando a série temporal
serie_temporal_normalizada = SeriesProcessor.normalizar_serie(serie_temporal)
print(f"Shape da série após normalização: {serie_temporal_normalizada.shape}")

# Gerando janelas temporais (features X e target Y)
X, Y = SeriesProcessor.criar_janela_temporal(serie_temporal_normalizada, lags)
print(f"Shape dos dados de entrada (X): {X.shape}")
print(f"Shape dos dados de saída (Y): {Y.shape}")

# Definindo tamanho do conjunto inicial de treinamento (por exemplo, 20% dos dados)
initial_size = int(0.2 * len(X))
print(f"Tamanho do conjunto inicial de treinamento: {initial_size} amostras")

# Dividindo os dados em conjunto inicial e streaming
X_init = X[:initial_size]
y_init = Y[:initial_size]
X_stream = X[initial_size:]
Y_stream = Y[initial_size:]

print(f"Shape do X_init: {X_init.shape}")
print(f"Shape do Y_init: {y_init.shape}")
print(f"Shape do X_stream: {X_stream.shape}")
print(f"Shape do Y_stream: {Y_stream.shape}")

# Alternativamente, você pode definir um número fixo de amostras iniciais
# Por exemplo, 100 primeiras amostras para treinamento
fixed_initial_size = 100
X_init_fixed = X[:fixed_initial_size]
Y_init_fixed = Y[:fixed_initial_size]
X_stream_fixed = X[fixed_initial_size:]
Y_stream_fixed = Y[fixed_initial_size:]

print("\nCom tamanho fixo:")
print(f"Shape do X_init_fixed: {X_init_fixed.shape}")
print(f"Shape do Y_init_fixed: {Y_init_fixed.shape}")
print(f"Shape do X_stream_fixed: {X_stream_fixed.shape}")
print(f"Shape do Y_stream_fixed: {Y_stream_fixed.shape}")

Processando série: ^BVSP
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Shape da série após normalização: (1244, 1)
Shape dos dados de entrada (X): (1239, 5)
Shape dos dados de saída (Y): (1239, 1)
Tamanho do conjunto inicial de treinamento: 247 amostras
Shape do X_init: (247, 5)
Shape do Y_init: (247, 1)
Shape do X_stream: (992, 5)
Shape do Y_stream: (992, 1)

Com tamanho fixo:
Shape do X_init_fixed: (100, 5)
Shape do Y_init_fixed: (100, 1)
Shape do X_stream_fixed: (1139, 5)
Shape do Y_stream_fixed: (1139, 1)


In [7]:
# 2. Dados Reais de Séries Financeiras
def preparar_dados_financeiros(ticker="^BVSP", periodo="5y", lags=10):
    """Prepara dados de séries financeiras para experimentos"""
    print(f"Baixando dados para {ticker} nos últimos {periodo}...")

    # Baixar dados
    serie_temporal = SeriesProcessor.baixar_dados(ticker, periodo)

    # Pré-processamento
    serie_normalizada = SeriesProcessor.normalizar_serie(serie_temporal)

    # Criar janelas temporais
    X, y = SeriesProcessor.criar_janela_temporal(serie_normalizada, lags)

    print(f"Dados processados: {X.shape[0]} amostras com {X.shape[1]} features")

    return X, y, serie_temporal

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tqdm


def executar_framework(X, y,
                      detector_tipo="DDM",
                      tamanho_inicial=200,
                      tamanho_janela=50,
                      desempenho_minimo=0.5,
                      modelo_base=None):
    """
    Executa o framework de detecção e adaptação em um conjunto de dados

    Parâmetros:
    -----------
    X : ndarray
        Features de entrada
    y : ndarray
        Valores alvo
    detector_tipo : str
        Tipo de detector a ser usado ("DDM", "ADWIN", "ECDD")
    tamanho_inicial : int
        Número de amostras para treinamento inicial
    tamanho_janela : int
        Tamanho da janela de dados recentes
    desempenho_minimo : float
        Limiar mínimo de desempenho para manter um modelo
    modelo_base : object
        Modelo de base para inicializar (se None, usa LinearRegression)

    Retorna:
    --------
    dict
        Resultados do experimento com métricas e estados
    """
    # Dividir dados
    X_init, y_init = X[:tamanho_inicial], y[:tamanho_inicial]
    X_stream, y_stream = X[tamanho_inicial:], y[tamanho_inicial:]

    print(f"Treinamento inicial: {len(X_init)} amostras, Stream: {len(X_stream)} amostras")

    # Inicializar modelos
    pool_modelos = FrameworkDetector.treinar_modelos_iniciais(X_init, y_init)
    modelo_atual = pool_modelos[0]  # Começa com o primeiro modelo

    print(f"Pool inicial contém {len(pool_modelos)} modelos")

    # Inicializar detector
    if detector_tipo == "DDM":
        detector = DDMDetector()
    elif detector_tipo == "ADWIN":
        detector = ADWINDetector()
    else:
        raise ValueError(f"Detector '{detector_tipo}' não suportado")

    # Estruturas para armazenar resultados
    janela_dados_recentes = []

    # Métricas
    erros_predicao = []
    estados_detector = []
    pontos_drift = []
    metricas_rmse = []
    metricas_mae = []
    metricas_r2 = []

    # Processamento do stream
    print("Processando stream de dados...")
    for i, (x_t, y_t) in enumerate(tqdm(zip(X_stream, y_stream), total=len(X_stream))):
        # Adaptar formato para previsão
        x_t_reshaped = x_t.reshape(1, -1)

        # 1. Predição
        try:
            y_pred = modelo_atual.prever(x_t_reshaped)[0]
        except Exception as e:
            print(f"Erro na predição: {e}")
            y_pred = np.mean(y_init)  # Fallback para média dos dados iniciais

        # 2. Cálculo do erro e atualização do detector
        erro = abs(y_t - y_pred)
        detector.atualizar(erro)

        # 3. Obter estado do detector
        estado = FrameworkDetector.get_state(detector)  # 'NORMAL', 'ALERTA', 'MUDANÇA'

        # 4. Ação baseada no estado
        if estado == "NORMAL":
            # Apenas armazena os dados recentes
            janela_dados_recentes = FrameworkDetector.adicionar_a_janela(janela_dados_recentes, (x_t, y_t), tamanho_janela)

        elif estado == "ALERTA":
            # Adapta o modelo se suportar aprendizado incremental
            if hasattr(modelo_atual, "partial_fit"):
                modelo_atual.partial_fit(x_t_reshaped, np.array([y_t]))
            janela_dados_recentes = FrameworkDetector.adicionar_a_janela(janela_dados_recentes, (x_t, y_t), tamanho_janela)

        elif estado == "MUDANÇA":
            # Registra o ponto de drift
            ponto_drift = tamanho_inicial + i
            pontos_drift.append(ponto_drift)
            print(f"Drift detectado no ponto {ponto_drift}")

            # Adiciona o modelo atual ao pool
            pool_modelos.append(np.copy.deepcopy(modelo_atual))

            # Seleciona o melhor modelo do pool
            melhor_modelo = FrameworkDetector.selecionar_melhor_modelo(pool_modelos, janela_dados_recentes)

            # Avalia o desempenho do melhor modelo na janela recente
            desempenho = FrameworkDetector.desempenho(melhor_modelo, janela_dados_recentes)
            mse = mean_squared_error([y for _, y in janela_dados_recentes],
                               melhor_modelo.prever([x for x, _ in janela_dados_recentes]))

            # Se o desempenho for adequado, usa o melhor modelo do pool
            if desempenho <= desempenho_minimo:  # Menor MSE é melhor
                modelo_atual = melhor_modelo
                print(f"Usando modelo do pool com MSE: {mse:.4f}")
            else:
                # Caso contrário, treina um novo modelo com dados recentes
                print("Treinando novo modelo...")
                if modelo_base is None:
                    novo_modelo = LinearRegression()
                else:
                    novo_modelo = copy.deepcopy(modelo_base)

                # Extrai dados da janela
                X_recente = np.array([x for x, _ in janela_dados_recentes])
                y_recente = np.array([y for _, y in janela_dados_recentes])

                # Treina o novo modelo
                try:
                    novo_modelo.fit(X_recente, y_recente)
                    modelo_atual = FrameworkDetector.criar_modelo_wrapper(novo_modelo)
                except Exception as e:
                    print(f"Erro ao treinar novo modelo: {e}")
                    # Mantém o melhor modelo do pool
                    modelo_atual = melhor_modelo

            # Limpa a janela
            janela_dados_recentes = [(x_t, y_t)]

        # Armazenar métricas e estados
        erros_predicao.append(erro)
        estados_detector.append(estado)

        # Calcular métricas de desempenho periodicamente
        if i % 50 == 0 and len(janela_dados_recentes) > 5:
            X_janela = np.array([x for x, _ in janela_dados_recentes])
            y_janela = np.array([y for _, y in janela_dados_recentes])
            y_prev = modelo_atual.prever(X_janela)

            try:
                rmse = np.sqrt(mean_squared_error(y_janela, y_prev))
                mae = mean_absolute_error(y_janela, y_prev)
                r2 = r2_score(y_janela, y_prev)

                metricas_rmse.append((tamanho_inicial + i, rmse))
                metricas_mae.append((tamanho_inicial + i, mae))
                metricas_r2.append((tamanho_inicial + i, r2))
            except Exception as e:
                print(f"Erro ao calcular métricas: {e}")

    # Preparar resultados
    resultados = {
        "detector": detector_tipo,
        "pool_modelos": pool_modelos,
        "modelo_final": modelo_atual,
        "erros_predicao": erros_predicao,
        "estados_detector": estados_detector,
        "pontos_drift": pontos_drift,
        "metricas_rmse": metricas_rmse,
        "metricas_mae": metricas_mae,
        "metricas_r2": metricas_r2
    }

    return resultados

# **2. Configurações Iniciais**

In [3]:
modelo_escolhido = "Ridge"     # ou "Ridge", "Tree"
detector_escolhido = "DDM"  # ou "DDM", "ADWIN"
desempenho_minimo = 0.05     # Define quando criar novo modelo

# **3. Inicialização**

In [4]:
pool_modelos = FrameworkDetector.treinar_modelos_iniciais(X=X_init, y=y_init)
modelo_atual = pool_modelos[0]

# Detector dinâmico
if detector_escolhido == "DDM":
    detector = DDMDetector()
elif detector_escolhido == "ADWIN":
    detector = ADWINDetector()

janela_dados_recentes = []

/home/liedson/.pyenv/versions/3.12.3/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/liedson/.pyenv/versions/3.12.3/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
# Loop principal de detecção e adaptação
from numpy import copy

for x_t, y_t in zip(X_stream, Y_stream):

    # 1. Predição
    y_pred = modelo_atual.prever([x_t])[0]

    # 2. Cálculo do erro
    erro = abs(y_t - y_pred)
    detector.atualizar(erro)

    # 3. Estado do detector
    estado = FrameworkDetector.get_state(detector)  # 'NORMAL', 'ALERTA', 'MUDANÇA'

    # 4. Ação baseada no estado
    if estado == "NORMAL":
        janela_dados_recentes.append((x_t, y_t))

    elif estado == "ALERTA":
        if hasattr(modelo_atual, "partial_fit"):
            modelo_atual.partial_fit([x_t], [y_t])
        janela_dados_recentes.append((x_t, y_t))

    elif estado == "MUDANÇA":
        # Salva uma cópia profunda do modelo atual no pool
        pool_modelos.append(copy.deepcopy(modelo_atual))

        # Seleciona o melhor modelo do pool
        melhor_modelo = FrameworkDetector.selecionar_melhor_modelo(pool_modelos, janela_dados_recentes)

        if FrameworkDetector.desempenho(melhor_modelo, janela_dados_recentes) >= desempenho_minimo:
            modelo_atual = melhor_modelo
        else:
            modelo_atual = FrameworkDetector.criar_modelo(modelo_escolhido)
            X_novo = [x for x, _ in janela_dados_recentes]
            y_novo = [y for _, y in janela_dados_recentes]
            modelo_atual.fit(X_novo, y_novo)

        janela_dados_recentes = []
